In [1]:
'''
# Sample code to perform I/O:

name = input()                  # Reading input from STDIN
print('Hi, %s.' % name)         # Writing output to STDOUT

# Warning: Printing unwanted or ill-formatted data to output will cause the test cases to fail
'''

# Write your code here
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBClassifier


train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

data_cat=train.select_dtypes(exclude=[np.number])
data_num=train.select_dtypes(include=[np.number])

sc=StandardScaler()
data_num_sc=sc.fit_transform(data_num[[ 'admission_type_id', 'discharge_disposition_id', 
                          'admission_source_id', 'time_in_hospital', 'tel_3', 'tel_4', 'tel_5', 'tel_6', 'tel_7', 'tel_8', 'tel_12']])
data_num_sc_df=pd.DataFrame(data_num_sc,columns=['admission_type_id', 'discharge_disposition_id', 'admission_source_id', 
                                                 'time_in_hospital', 'tel_3', 'tel_4', 'tel_5', 'tel_6', 'tel_7', 'tel_8', 'tel_12'])

data_cat.replace('?',np.nan,inplace=True)
data_cat.drop('weight',axis=1,inplace=True)
data_cat['tel_2'].replace(np.nan,'InternalMedicine',inplace=True)
data_cat['tel_1'].replace(np.nan,'MC',inplace=True)
data_cat['race'].replace(np.nan,'Caucasian',inplace=True)
data_cat.replace(np.nan,'0',inplace=True)
lb_en=LabelEncoder()
df_cat=data_cat.apply(lb_en.fit_transform)
df_cat_sc=sc.fit_transform(df_cat)
data_cat_sc=pd.DataFrame(df_cat_sc,columns=df_cat.columns)
df_train=pd.concat([data_num_sc_df,data_cat_sc],axis=1)
X=df_train
y=train['diabetesMed']

xgb_mdl = XGBClassifier(learning_rate = 0.2, max_depth = 4, n_estimators = 100, subsample = 0.9)

xgb_mdl.fit(X, y)

test.replace('?',np.nan,inplace=True)
test.drop('weight',axis=1,inplace=True)

test['tel_2'].replace(np.nan,'InternalMedicine',inplace=True)
test['tel_1'].replace(np.nan,'MC',inplace=True)
test['race'].replace(np.nan,'Caucasian',inplace=True)
test.replace(np.nan,'0',inplace=True)
cat_cols=test.select_dtypes(exclude=[np.number]).columns
num_cols=test.select_dtypes(include=[np.number]).columns

test_df_cat=test[cat_cols].apply(lb_en.fit_transform)
test_df=pd.concat([test[num_cols],test_df_cat],axis=1)
test_df_sc=sc.fit_transform(test_df)
test_sc=pd.DataFrame(test_df_sc,columns=test_df.columns)
X_test=test_sc.drop(['encounter_id','patient_id'],axis=1)

y_pred_final=xgb_mdl.predict(X_test)
submission=pd.DataFrame({'encounter_id':test['encounter_id'],'diabetesMed':y_pred_final})
print(submission.head())
submission.to_csv('submission_anwesh.csv',index=False)


C:\Users\91801\Anaconda3\lib\site-packages\pandas\core\frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)
C:\Users\91801\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Users\91801\Anaconda3\lib\site-packages\pandas\core\generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


   encounter_id  diabetesMed
0      97074168            1
1      38255910            1
2      36573714            0
3      73972230            1
4      68202342            0


In [2]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7970 entries, 0 to 7969
Data columns (total 2 columns):
encounter_id    7970 non-null int64
diabetesMed     7970 non-null int64
dtypes: int64(2)
memory usage: 124.6 KB


In [3]:
submission.shape

(7970, 2)

In [4]:
test.shape

(7970, 48)